In [1]:
import json
from numpy import array
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding



Using TensorFlow backend.


In [2]:
def create_tokenizer(questions):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(questions)
	return tokenizer

In [3]:
def clean_text(str):
    newstr = str.replace("'s", "")
    return newstr

In [4]:
vqa_train = json.load(open('C:/VQA/data/vqa_train.json'))
questions =[]
answers =[]
summary=[]
for i in range(len(vqa_train)):
    str = clean_text(vqa_train[i]['question'][:-1])
    ques = 'start '+str+' end'
    ans = vqa_train[i]['ans']
    questions.append(ques)
    answers.append(ans)
    summary.append(ques)
    summary.append(ans)
questions=questions[:1000]
answers = answers[:1000]
print(summary[0]+' '+summary[1])

start What is this photo taken looking through end net


In [5]:
tokenizer = create_tokenizer(summary)
# print(tokenizer.word_index)

{'end': 1, 'start': 2, 'the': 3, 'is': 4, 'what': 5, 'are': 6, 'this': 7, 'yes': 8, 'no': 9, 'in': 10, 'on': 11, 'of': 12, 'a': 13, 'how': 14, 'many': 15, 'color': 16, 'there': 17, 'man': 18, 'does': 19, 'people': 20, 'picture': 21, 'to': 22, 'white': 23, 'wearing': 24, 'it': 25, '1': 26, 'these': 27, '2': 28, 'where': 29, 'person': 30, 'have': 31, 'kind': 32, 'or': 33, 'photo': 34, 'do': 35, 'woman': 36, 'you': 37, 'red': 38, 'doing': 39, 'black': 40, 'type': 41, 'blue': 42, 'animal': 43, 'room': 44, 'dog': 45, 'and': 46, 'they': 47, 'cat': 48, 'be': 49, 'holding': 50, '3': 51, 'animals': 52, 'green': 53, 'train': 54, 'for': 55, 'water': 56, 'can': 57, 'right': 58, 'at': 59, 'food': 60, 'sign': 61, 'bus': 62, 'his': 63, 'left': 64, 'he': 65, 'shirt': 66, 'that': 67, 'which': 68, 'any': 69, 'playing': 70, 'see': 71, 'table': 72, 'pizza': 73, 'sitting': 74, 'plane': 75, 'an': 76, '0': 77, 'yellow': 78, 'made': 79, 'plate': 80, 'why': 81, 'brown': 82, 'shown': 83, 'standing': 84, '4': 85

In [6]:
# vocab=[]
# from nltk.tokenize import word_tokenize
# for que in questions:
#     tokenize_word = word_tokenize(que)
#     for word in tokenize_word:
#         vocab.append(word)
# for ans in answers:
#     tokenize_word = word_tokenize(ans)
#     for word in tokenize_word:
#         vocab.append(word)

        

In [7]:
#vocab = set(vocab)
vocab_len = len(tokenizer.word_index)+1
print(vocab_len)

19341


In [8]:
# embedded_questions = [one_hot(ques,vocab_len) for ques in questions]
# print(questions[0])
# print(questions[1])
# print(embedded_questions[0])
# print(embedded_questions[1])


# embedded_ans = [one_hot(ans,vocab_len) for ans in answers]
# print(embedded_ans[0])
# print(embedded_ans[1])
embedded_questions = [tokenizer.texts_to_sequences([ques])[0] for ques in questions]
print(questions[0])
print(embedded_questions[0])

embedded_ans = [tokenizer.texts_to_sequences([ans])[0] for ans in answers]
print(answers[0])
print(embedded_ans[0])

start What is this photo taken looking through end
[2, 5, 4, 7, 34, 89, 120, 546, 1]
net
[1147]


In [9]:
# question_count = lambda sentence: len(word_tokenize(sentence))
# longest_question = max(questions, key=question_count)
# length_long_question = len(word_tokenize(longest_question))
# print('Longest sentence ='+str(length_long_question))
# ans_count = lambda sentence: len(word_tokenize(sentence))
# longest_ans = max(answers, key=ans_count)
# length_long_ans = len(word_tokenize(longest_ans))
# print('Longest sentence ='+str(length_long_ans))

length_long_question = 34


In [10]:
padded_questions = pad_sequences(embedded_questions, length_long_question, padding='post')
print(padded_questions.shape)
print(padded_questions[0])
print(padded_questions[1])
padded_answers = pad_sequences(embedded_ans, vocab_len, padding='post')

#padded_answers =  [item[0] for item in padded_answers]
print(padded_answers.shape)

(1000, 34)
[  2   5   4   7  34  89 120 546   1   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  2   5 634   4   7  18  70   1   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
(1000, 19341)


In [11]:

model = Sequential()
model.add(Embedding(vocab_len,32,input_length=length_long_question))
model.add(LSTM(256,dropout=0.2))
#model.add(Flatten())
model.add(Dense(vocab_len,activation='sigmoid'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 34, 32)            618912    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               295936    
_________________________________________________________________
dense_1 (Dense)              (None, 19341)             4970637   
Total params: 5,885,485
Trainable params: 5,885,485
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
model.fit(padded_questions,padded_answers,epochs=100,verbose=1)


Epoch 1/100
 896/1000 [=========================>....] - ETA: 1s - loss: 0.2814 - acc: 0.9603

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(padded_questions, padded_answers, verbose=1)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
def word_for_id(integer,tokenizer):
	for word,index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [ ]:
import numpy as np
test_question =padded_questions[100].reshape((1,-1))
print(test_question[:1])
# test_question  =np.reshape(test_question,(1,))
print(test_question.shape)
test_output = model.predict(test_question[:1],verbose=1)
print(test_output)
test_output = np.argmax(test_output.reshape((-1,1)))
print(test_output)
test_answer = word_for_id(test_output,tokenizer)
print(questions[0])
print(test_answer)